In [1]:
import tensorflow as tf
K = tf.keras.backend
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names


DeepCTR version 0.9.3 detected. Your version is 0.9.1.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.3


In [11]:
data=pd.read_csv('../data/data_bert.csv',encoding='utf-8')

In [12]:
data.head()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,...,tag_bert758,tag_bert759,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767
0,0,207000097,24696418,1.0,5,0,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
1,1,207000097,32507861,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
2,2,207000097,39060478,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
3,3,207000097,85909646,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
4,4,207000097,172874760,1.0,5,0,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011


In [4]:
data.tail()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,...,tag_bert758,tag_bert759,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767
845459,845459,507078823,9873156,0.440,6,0,0,12,0.999862,0.999886,...,0.974894,0.995783,-0.989222,0.999992,0.94585,-0.997905,-0.995735,-0.999130,-0.994474,0.993043
845460,845460,764155292,85274975,0.232,5,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845461,845461,764155292,51463228,0.000,6,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845462,845462,764155292,693496460,0.005,3,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845463,845463,718667726,346747239,0.753,4,0,1,57,0.998748,0.999991,...,0.980760,0.997610,-0.997252,0.999985,0.96784,-0.996740,-0.999278,-0.996762,-0.996212,0.990695


In [13]:
data=data.drop(['Unnamed: 0'],axis=1)

In [14]:
data['user_nm']=0.
data['dance_nm']=0.

In [15]:
from tqdm import tqdm

In [16]:
#user_nm赋值
k = 10000/(data["user_id"].max()-data["user_id"].min())
for i in tqdm(range(len(data))):
    
    transform_value=k*(data["user_id"][i]-data["user_id"].min()) 
    data["user_nm"][i]=transform_value


  0%|                                                                                       | 0/845464 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_10452\3739655352.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["user_nm"][i]=transform_value
100%|████████████████████████████████████████████████████████████████████████| 845464/845464 [2:53:23<00:00, 81.27it/s]


In [17]:
#dance_nm赋值
k1 = 10000/(data["dance_id"].max()-data["dance_id"].min())
for i in tqdm(range(len(data))):
    transform_value=k1*(data["dance_id"][i]-data["dance_id"].min())+1 
#     print(transform_value)
    data["dance_nm"][i]=transform_value

  0%|                                                                                       | 0/845464 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_10452\3916622010.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["dance_nm"][i]=transform_value
100%|████████████████████████████████████████████████████████████████████████| 845464/845464 [2:56:02<00:00, 80.04it/s]


In [19]:
data.to_csv("data_bert_nm.csv",encoding='utf-8')

In [ ]:
cols = data.columns

In [ ]:
cols

In [7]:
len([column for column in data][7:])

1536

In [2]:
data=pd.read_csv("../data/data_nm.csv",encoding='utf-8')

In [3]:
data.head()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_em0,title_em1,...,tag_em292,tag_em293,tag_em294,tag_em295,tag_em296,tag_em297,tag_em298,tag_em299,user_nm,dance_nm
0,0,207000097,24696418,1.0,5,0,27,3914,0.229985,0.410569,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,115.001742,1.0
1,1,207000097,32507861,1.0,6,2,27,3914,0.229985,0.410569,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,151.376687,1.0
2,2,207000097,39060478,1.0,6,2,27,3914,0.229985,0.410569,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,181.889753,1.0
3,3,207000097,85909646,1.0,6,2,27,3914,0.229985,0.410569,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,400.048661,1.0
4,4,207000097,172874760,1.0,5,0,27,3914,0.229985,0.410569,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,805.012401,1.0


In [4]:
data=data.drop(['Unnamed: 0'],axis=1)

In [5]:
data

,dance_id,user_id,lable,level,sex_em,favorites,play,title_em0,title_em1,title_em2,...,tag_em292,tag_em293,tag_em294,tag_em295,tag_em296,tag_em297,tag_em298,tag_em299,user_nm,dance_nm
0,207000097,24696418,1.000,5,0,27,3914,0.229985,0.410569,0.608845,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,115.001742,1.000000
1,207000097,32507861,1.000,6,2,27,3914,0.229985,0.410569,0.608845,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,151.376687,1.000000
2,207000097,39060478,1.000,6,2,27,3914,0.229985,0.410569,0.608845,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,181.889753,1.000000
3,207000097,85909646,1.000,6,2,27,3914,0.229985,0.410569,0.608845,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,400.048661,1.000000
4,207000097,172874760,1.000,5,0,27,3914,0.229985,0.410569,0.608845,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,805.012401,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845459,507078823,9873156,0.440,6,0,0,12,-0.436733,0.155030,0.830092,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,45.975401,3893.739553
845460,764155292,85274975,0.232,5,0,0,19,-0.593119,0.373108,0.980985,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,397.093237,7228.636873
845461,764155292,51463228,0.000,6,0,0,19,-0.593119,0.373108,0.980985,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,239.644682,7228.636873
845462,764155292,693496460,0.005,3,0,0,19,-0.593119,0.373108,0.980985,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3229.351326,7228.636873


In [6]:
#划分特征
sparse_cols= ['dance_nm','user_nm','level','sex_em','favorites','play']
dense_cols= [column for column in data][7:-2]

In [7]:
len(dense_cols)

600

In [9]:
for i in dense_cols:
    print(i)

title_em0
title_em1
title_em2
title_em3
title_em4
title_em5
title_em6
title_em7
title_em8
title_em9
title_em10
title_em11
title_em12
title_em13
title_em14
title_em15
title_em16
title_em17
title_em18
title_em19
title_em20
title_em21
title_em22
title_em23
title_em24
title_em25
title_em26
title_em27
title_em28
title_em29
title_em30
title_em31
title_em32
title_em33
title_em34
title_em35
title_em36
title_em37
title_em38
title_em39
title_em40
title_em41
title_em42
title_em43
title_em44
title_em45
title_em46
title_em47
title_em48
title_em49
title_em50
title_em51
title_em52
title_em53
title_em54
title_em55
title_em56
title_em57
title_em58
title_em59
title_em60
title_em61
title_em62
title_em63
title_em64
title_em65
title_em66
title_em67
title_em68
title_em69
title_em70
title_em71
title_em72
title_em73
title_em74
title_em75
title_em76
title_em77
title_em78
title_em79
title_em80
title_em81
title_em82
title_em83
title_em84
title_em85
title_em86
title_em87
title_em88
title_em89
title_em90
title_em9

In [10]:
for feat in sparse_cols:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat],)

In [11]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1,embedding_dim=4)
                       for i,feat in enumerate(sparse_cols)] + [DenseFeat(feat, 1,)
                      for feat in dense_cols]

In [12]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [12]:
#数据构建
train_data = data.sample(frac=1)
train_data.head()

,dance_id,user_id,lable,level,sex_em,favorites,play,title_em0,title_em1,title_em2,...,tag_em292,tag_em293,tag_em294,tag_em295,tag_em296,tag_em297,tag_em298,tag_em299,user_nm,dance_nm
356885,252249329,33595300,1.00,6,0,1110,7308,-0.296494,0.130040,0.607456,...,-0.427353,-0.202041,0.754443,0.893175,0.480693,0.438089,0.245608,-0.020120,62517,3708
546370,634038919,488139629,0.00,5,1,11,1424,0.192538,0.488864,-0.258159,...,0.342735,-0.068894,0.651860,1.037736,0.382270,0.052429,0.543151,-0.529125,191518,21524
281723,378914077,596472085,1.00,5,0,11,7961,-1.093407,-0.512780,0.258357,...,-0.563499,0.029256,0.494778,0.698658,0.484723,0.127818,0.215772,-0.011090,207170,9162
117892,719419157,523913726,0.28,3,1,228,3484,-0.800382,0.221250,1.224247,...,0.347509,-0.512089,0.856844,0.086465,0.394723,-0.104743,0.770650,0.183358,202548,25977
29486,590103861,234338844,1.00,4,0,118,4508,-0.262465,-0.123052,0.697224,...,-0.154432,0.706104,0.758894,0.543305,1.201806,0.140227,0.024433,0.016917,115184,18600


In [14]:
train, test = train_test_split(data, test_size=0.2,random_state=2022)

In [15]:
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [16]:
model = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary')
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

In [17]:
history = model.fit(train_model_input, train[['lable']].values,
                    batch_size=512, epochs=100, verbose=1, validation_split=0.2)

Epoch 1/100
1057/1057 [==============================] - 50s 43ms/step - loss: 0.4593 - binary_crossentropy: 0.4561 - val_loss: 0.4433 - val_binary_crossentropy: 0.4375
Epoch 2/100
1057/1057 [==============================] - 42s 40ms/step - loss: 0.4189 - binary_crossentropy: 0.4094 - val_loss: 0.4524 - val_binary_crossentropy: 0.4404
Epoch 3/100
1057/1057 [==============================] - 42s 40ms/step - loss: 0.3846 - binary_crossentropy: 0.3695 - val_loss: 0.4871 - val_binary_crossentropy: 0.4699
Epoch 4/100
1057/1057 [==============================] - 43s 40ms/step - loss: 0.3538 - binary_crossentropy: 0.3348 - val_loss: 0.5189 - val_binary_crossentropy: 0.4981
Epoch 5/100
1057/1057 [==============================] - 42s 40ms/step - loss: 0.3305 - binary_crossentropy: 0.3088 - val_loss: 0.5519 - val_binary_crossentropy: 0.5288
Epoch 6/100
1057/1057 [==============================] - 43s 41ms/step - loss: 0.3102 - binary_crossentropy: 0.2867 - val_loss: 0.5880 - val_binary_crossen

1057/1057 [==============================] - 42s 40ms/step - loss: 0.2036 - binary_crossentropy: 0.1802 - val_loss: 0.8927 - val_binary_crossentropy: 0.8691
Epoch 50/100
1057/1057 [==============================] - 42s 40ms/step - loss: 0.2028 - binary_crossentropy: 0.1796 - val_loss: 0.8505 - val_binary_crossentropy: 0.8270
Epoch 51/100
1057/1057 [==============================] - 42s 40ms/step - loss: 0.2029 - binary_crossentropy: 0.1797 - val_loss: 0.9041 - val_binary_crossentropy: 0.8806
Epoch 52/100
1057/1057 [==============================] - 42s 40ms/step - loss: 0.2024 - binary_crossentropy: 0.1792 - val_loss: 0.9002 - val_binary_crossentropy: 0.8768
Epoch 53/100
1057/1057 [==============================] - 44s 41ms/step - loss: 0.2023 - binary_crossentropy: 0.1792 - val_loss: 0.8731 - val_binary_crossentropy: 0.8497
Epoch 54/100
1057/1057 [==============================] - 43s 40ms/step - loss: 0.2022 - binary_crossentropy: 0.1792 - val_loss: 0.9002 - val_binary_crossentropy: 

1057/1057 [==============================] - 42s 39ms/step - loss: 0.1924 - binary_crossentropy: 0.1728 - val_loss: 0.8823 - val_binary_crossentropy: 0.8624
Epoch 98/100
1057/1057 [==============================] - 41s 39ms/step - loss: 0.1932 - binary_crossentropy: 0.1736 - val_loss: 0.8956 - val_binary_crossentropy: 0.8756
Epoch 99/100
1057/1057 [==============================] - 41s 39ms/step - loss: 0.1921 - binary_crossentropy: 0.1725 - val_loss: 0.9003 - val_binary_crossentropy: 0.8804
Epoch 100/100
1057/1057 [==============================] - 42s 39ms/step - loss: 0.1918 - binary_crossentropy: 0.1722 - val_loss: 0.8603 - val_binary_crossentropy: 0.8405


In [15]:
# history = model.fit(train_model_input, train[['lable']].values,
#                     batch_size=256, epochs=100, verbose=2, validation_split=0.2, )
from keras.callbacks import *
# 回调函数
file_path = "deepfm_w2v.h5"
earlystopping = EarlyStopping(monitor="val_loss", patience=3)
checkpoint = ModelCheckpoint(
    file_path, save_weights_only=False, verbose=1, save_best_only=True)
callbacks_list = [earlystopping, checkpoint]

history = model.fit(train_model_input, train[['lable']].values, 
                   batch_size=256,
                  epochs=100,
                  validation_split=0.2,
                  callbacks=callbacks_list,
                  shuffle=True)
# history=model.fit(train_sparse_x+train_dense_x,
#                   train_label, 
#                   epochs=100,
#                   batch_size=32,
#                   validation_data=(valid_sparse_x+valid_dense_x, valid_label))

Epoch 1/100
2114/2114 [==============================] - 322s 149ms/step - loss: 0.4654 - binary_crossentropy: 0.4602 - val_loss: 0.4451 - val_binary_crossentropy: 0.4363

Epoch 00001: val_loss improved from inf to 0.44514, saving model to deepfm_bert.h5


D:\Anaconda3 2020.11\envs\tf-gpu-25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
2114/2114 [==============================] - 309s 146ms/step - loss: 0.4305 - binary_crossentropy: 0.4178 - val_loss: 0.4479 - val_binary_crossentropy: 0.4337

Epoch 00002: val_loss did not improve from 0.44514
Epoch 3/100
2114/2114 [==============================] - 310s 147ms/step - loss: 0.4079 - binary_crossentropy: 0.3916 - val_loss: 0.4685 - val_binary_crossentropy: 0.4509

Epoch 00003: val_loss did not improve from 0.44514
Epoch 4/100
2114/2114 [==============================] - 309s 146ms/step - loss: 0.3873 - binary_crossentropy: 0.3683 - val_loss: 0.5188 - val_binary_crossentropy: 0.4990

Epoch 00004: val_loss did not improve from 0.44514


In [ ]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [18]:
model.save('deepfm_w2v.h5')


D:\Anaconda2021.11\envs\tf-gpu25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
